# Hello World! (TEST module)

This notebook is intended to show how to use the Teselagen Python's client to access the TEST module. 

In [1]:
import json
import platform
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

from teselagen.api.test_client import TESTClient
from teselagen.utils.utils import get_project_root

plt.style.use('seaborn-colorblind')

print(f"python version     : {platform.python_version()}")
print(f"json version       : {json.__version__}")
print(f"pandas version     : {pd.__version__}")
print(f"matplotlib version : {matplotlib.__version__}")

python version     : 3.6.9
json version       : 2.0.9
pandas version     : 1.1.2
matplotlib version : 3.3.2


## Connect and Login

In [2]:
host_url="https://platform.teselagen.com"
client = TESTClient(host_url=host_url)
# The following command will promt you to type username (email) and password
client.login()
# Alternatively you can add credential information in the arguments
#client.login(username="your@email", password="ZZZ")

Connection Accepted


## Laboratories

Laboratories are used to separate data access into groups of users. When you add an experiment into a lab it will only be seen by users that have access to that Lab. 

At the next cell a lab is selected. The remaining actions of this notebook will be done inside that lab.

In [3]:
## Select a Laboratory
client.select_laboratory(lab_name='Data Science Team')

Selected Lab: Data Science Team


## Experiments

Experiments represents all the studies inside a laboratory. Each experiment can contain several Assays. The **get_experiments** function returns a list of dictionaries that contain experiments information.

In [4]:
## Get My Experiments from the selected Laboratory.
experiments = client.get_experiments()
# Print some of them
for exp in experiments: print(exp)

{'id': '37', 'name': 'TD Experiment'}
{'id': '38', 'name': 'TD Glucose Production Experiment'}
{'id': '40', 'name': 'API Experiment'}
{'id': '46', 'name': 'New Experiment'}
{'id': '47', 'name': 'New Experiment 2'}
{'id': '169', 'name': 'Isoprenol Production'}


In [5]:
## Create a new Experiment in the selected Laboratory.
new_experiment = client.create_experiment(experiment_name="Jupyter Experiment")
print(new_experiment)

{'id': '231', 'name': 'Jupyter Experiment'}


## Assays

The assay object allows you to group data obtained from the same *protocol*.

Likewise other getters, the get_assays output can be easily parsed into a pandas dataframe. Here we show you how:

In [6]:
## Get All Assays From all experiments in the selected Laboratory
assays_dataframe = pd.DataFrame(client.get_assays())
# Or, alternatevily, if you want to use id as index in the dataframe
#assays_dataframe = pd.DataFrame.from_records(client.get_assays(), index='id')
display(assays_dataframe)

,id,name,experiment
0,6,TD Assay 2,"{'id': '37', 'name': 'TD Experiment'}"
1,7,TD Glucose Production Assay,"{'id': '38', 'name': 'TD Glucose Production Ex..."
2,8,TD Glucose Production Assay 2,"{'id': '38', 'name': 'TD Glucose Production Ex..."
3,101,Example Assay,"{'id': '40', 'name': 'API Experiment'}"
4,111,new assay,"{'id': '46', 'name': 'New Experiment'}"
5,112,New Assay,"{'id': '46', 'name': 'New Experiment'}"
6,113,API Assay,"{'id': '40', 'name': 'API Experiment'}"
7,128,Jupyter Assay,"{'id': '37', 'name': 'TD Experiment'}"
8,129,Jupyter Assay,"{'id': '37', 'name': 'TD Experiment'}"
9,132,Jupyter Assay,"{'id': '37', 'name': 'TD Experiment'}"


You can also create assays using the client

In [7]:
## Create a new Assay within a new experiment
new_assay = client.create_assay(experiment_id=new_experiment['id'], assay_name='Jupyter Assay')
print(new_assay)

{'id': '220', 'name': 'Jupyter Assay', 'experiment': {'id': '231', 'name': 'Jupyter Experiment'}}


And you can list all assays from an experiment

In [8]:
## Get All Assays From Experiment with ID="experiment_id"
assays = pd.DataFrame(client.get_assays(experiment_id=new_experiment['id'])) 
display(assays)

,id,name,experiment
0,220,Jupyter Assay,"{'id': '231', 'name': 'Jupyter Experiment'}"


## Files

The TEST module can also store files. Files are shared between users of the same lab and can be associated to specific assays.

In [9]:
## Get All Files info from the selected Laboratory
pd.DataFrame(client.get_files_info()).head()
## You can fetch info from a specific assay using the assay_id argument
#display(pd.DataFrame(client.get_files(assay_id=new_assay['id'])))

,id,name,assay
0,1,glucose_production_rate.csv,None
1,2,glucose_production_rate.csv,None
2,3,glucose_production_rate.csv,None
3,4,glucose_production_rate.csv,None
4,5,glucose_production_rate.csv,None


In [10]:
## Upload a File with an Assay.
filepath = str(get_project_root() / "teselagen/examples/test_module_example.csv")
new_file_info = client.upload_file(filepath=filepath, assay_id=new_assay['id'])
## Upload a File without an Assay.
#new_file_info = client.upload_file(filepath="./test_module_example.csv")
print(new_file_info)

{'id': '152', 'name': 'test_module_example.csv', 'assay': {'id': '220', 'name': 'Jupyter Assay'}}


In [11]:
## Gets File data contens
binarydata = client.download_file(file_id=new_file_info['id'])
df_file = pd.read_csv(binarydata, sep=",")
df_file.head()

,Line,Teselagen Example Descriptor 1,Teselagen Example Descriptor 2,Teselagen Example Target,Teselagen Example Target Metric
0,1,A0,B1,1,ug/mL
1,2,A0,B2,2,ug/mL
2,3,A0,B3,3,ug/mL
3,4,A0,B5,5,ug/mL
4,5,A0,B6,6,ug/mL


## Delete elements

You can delete an experiment to remove all its assays. Files work differently and you need to explicitly delete them. In the next cells we will download each object specifically:

In [12]:
## Deletes a File
client.delete_file(file_id=new_file_info['id'])

Deletion successful.


In [13]:
## Delete an Assay
client.delete_assay(assay_id=new_assay['id'])

Deletion successful.


In [14]:
## Delete an Experiment from the selected laboratory.
client.delete_experiment(experiment_id=new_experiment['id'])

Deletion successful.


In [15]:
# Delete all experiments with the name Jupyter Experiment 
# (if any remains due to uncompleted executions of the notebook)
# experiments = client.get_experiments()
# for exp in experiments:
#     if exp['name'] == "Jupyter Experiment":
#         client.delete_experiment(experiment_id=exp['id'])